# In this Notebook, I calculate various quantities related to inverse distance and combine all data in one excel file. 

In [60]:
import pandas as pd
import numpy as np
from geopy import distance

In [61]:
MA_district_coordinates = pd.read_excel('data/MA_AP_performance/MA_district_coordinates.xlsx').sort_values(by='District Code').reset_index(drop=True)

uni_coord_R1R2 = pd.read_excel('data/MA_AP_performance/hd2023_coordinate/hd2023_R1R2_data_MA_allnearby.xlsx').sort_values(by='INSTNM').reset_index(drop=True)
uni_coord_Public = pd.read_excel('data/MA_AP_performance/hd2023_coordinate/hd2023_Public_university_data_MA_allnearby.xlsx').sort_values(by='INSTNM').reset_index(drop=True)
uni_coord_Private_nfp = pd.read_excel('data/MA_AP_performance/hd2023_coordinate/hd2023_private_notforprofit_data_MA_allnearby.xlsx').sort_values(by='INSTNM').reset_index(drop=True)
uni_coord_Land_Grant = pd.read_excel('data/MA_AP_performance/hd2023_coordinate/hd2023_Land_Grant_data_MA_allnearby.xlsx').sort_values(by='INSTNM').reset_index(drop=True)
uni_coord_STEM = pd.read_excel('data/MA_AP_performance/hd2023_coordinate/hd2023_STEM_data_MA_allnearby.xlsx').sort_values(by='INSTNM').reset_index(drop=True)


###########################

def distance_district_univeristy_R1R2(district_code, university_name):
    district_coor = MA_district_coordinates[MA_district_coordinates['District Code'] == district_code][['LONGITUD', 'LATITUDE']].to_numpy()[0][::-1]
    university_coor = uni_coord_R1R2[uni_coord_R1R2['INSTNM'] == university_name][['LONGITUD', 'LATITUDE']].to_numpy()[0][::-1]
    return distance.distance(district_coor, university_coor).miles    ## return unit is miles

def distance_district_univeristy_Public(district_code, university_name):
    district_coor = MA_district_coordinates[MA_district_coordinates['District Code'] == district_code][['LONGITUD', 'LATITUDE']].to_numpy()[0][::-1]
    university_coor = uni_coord_Public[uni_coord_Public['INSTNM'] == university_name][['LONGITUD', 'LATITUDE']].to_numpy()[0][::-1]
    return distance.distance(district_coor, university_coor).miles    ## return unit is miles

def distance_district_univeristy_Private_nfp(district_code, university_name):
    district_coor = MA_district_coordinates[MA_district_coordinates['District Code'] == district_code][['LONGITUD', 'LATITUDE']].to_numpy()[0][::-1]
    university_coor = uni_coord_Private_nfp[uni_coord_Private_nfp['INSTNM'] == university_name][['LONGITUD', 'LATITUDE']].to_numpy()[0][::-1]
    return distance.distance(district_coor, university_coor).miles    ## return unit is miles

def distance_district_univeristy_Land_Grant(district_code, university_name):
    district_coor = MA_district_coordinates[MA_district_coordinates['District Code'] == district_code][['LONGITUD', 'LATITUDE']].to_numpy()[0][::-1]
    university_coor = uni_coord_Land_Grant[uni_coord_Land_Grant['INSTNM'] == university_name][['LONGITUD', 'LATITUDE']].to_numpy()[0][::-1]
    return distance.distance(district_coor, university_coor).miles    ## return unit is miles

def distance_district_univeristy_STEM(district_code, university_name):
    district_coor = MA_district_coordinates[MA_district_coordinates['District Code'] == district_code][['LONGITUD', 'LATITUDE']].to_numpy()[0][::-1]
    university_coor = uni_coord_STEM[uni_coord_STEM['INSTNM'] == university_name][['LONGITUD', 'LATITUDE']].to_numpy()[0][::-1]
    return distance.distance(district_coor, university_coor).miles    ## return unit is miles


###########################

epsilon_soften_factor = 10.0  ## [miles]

def inverse_distance_weight(distance_miles):
    return 1.0 / ( (distance_miles/epsilon_soften_factor) + 1 )

def enrollment_inverse_distance_weight(distance_miles, num_annual_enrollment):
    ## num_annual_enrollment is the number of annual enrollment
    return int(num_annual_enrollment) * 1.0 / ( (distance_miles/epsilon_soften_factor) + 1 )

def dormbed_inverse_distance_weight(distance_miles, num_dorm_bed):
    return int(num_dorm_bed) * 1.0 / ( (distance_miles/epsilon_soften_factor) + 1 )

# First, find the school districts that appear in all five year data

In [42]:
MA_distance_dict = pd.read_excel('data/MA_AP_performance/AP_performance_18_22.xlsx', sheet_name='20%s-%s'%(18, 18+1))
for year in [19, 20, 21, 22]:
    MA_distance_dict_next = pd.read_excel('data/MA_AP_performance/AP_performance_18_22.xlsx', sheet_name='20%s-%s'%(year, year+1))
    MA_distance_dict = MA_distance_dict[MA_distance_dict['District Code'].isin(MA_distance_dict['District Code'])]

MA_median_income = pd.read_excel('data/MA_AP_performance/MA_median_household_income.xlsx')
MA_district_population = pd.read_excel('data/MA_AP_performance/MA_district_population.xlsx')

MA_distance_dict = MA_distance_dict[MA_distance_dict['District Code'].isin(MA_district_coordinates['District Code'])]
MA_distance_dict = MA_distance_dict[MA_distance_dict['District Code'].isin(MA_median_income['District Code'])]
MA_distance_dict = MA_distance_dict[MA_distance_dict['District Code'].isin(MA_district_population['District Code'])]
MA_distance_dict = MA_distance_dict[MA_distance_dict['District Code'].isin(MA_expenditure_year['District Code'])]
MA_distance_dict = MA_distance_dict.sort_values(by='District Code').reset_index(drop=True)

#### delete redundant columns in the AP_performance excel sheet 
MA_distance_dict = MA_distance_dict.drop(columns=['Tests Taken', 'Score=1', 'Score=1', 'Score=2', 'Score=3', 'Score=4', 'Score=5', '% Score 1-2', '% Score 3-5'])
MA_distance_dict = MA_distance_dict.sort_values(by='District Code').reset_index(drop=True)

In [44]:
MA_distance_dict

,District Name,District Code
0,Abington,10000
1,Agawam,50000
2,Amesbury,70000
3,Andover,90000
4,Arlington,100000
...,...,...
217,Tantasqua,7700000
218,Triton,7730000
219,Wachusett,7750000
220,Quaboag Regional,7780000


# Second, calculate all distance-related quantities
### The following cell will take about 5 minutes to run

In [45]:
#### 1. Calculate distance to R1R2 universities
inv_distance_arr      = []
enll_inv_distance_arr = []
dorm_inv_distance_arr = []

for district_code in MA_distance_dict['District Code'].tolist():
    
    total_val = enll_total_val = dorm_total_val = 0
    
    for university_name in uni_coord_R1R2['INSTNM'].tolist():
        
        distance_ij           = distance_district_univeristy_R1R2(district_code, university_name)
        num_annual_enrollment = uni_coord_R1R2[uni_coord_R1R2['INSTNM'] == university_name]['Annual enrollment'].values[0]
        num_dorm_bed          = uni_coord_R1R2[uni_coord_R1R2['INSTNM'] == university_name]['Number of dorm beds'].values[0]
        
        total_val      += inverse_distance_weight( distance_ij )
        enll_total_val += enrollment_inverse_distance_weight( distance_ij, num_annual_enrollment )
        dorm_total_val += dormbed_inverse_distance_weight( distance_ij, num_dorm_bed)
        
    inv_distance_arr.append(total_val)
    enll_inv_distance_arr.append(enll_total_val)
    dorm_inv_distance_arr.append(dorm_total_val)
    
MA_distance_dict['Inverse Distance R1R2'] = inv_distance_arr
MA_distance_dict['Enrollment Inverse Distance R1R2'] = enll_inv_distance_arr
MA_distance_dict['Dorm Bed Inverse Distance R1R2'] = dorm_inv_distance_arr


#### 2. Calculate distance to Public universities
inv_distance_arr      = []
enll_inv_distance_arr = []
dorm_inv_distance_arr = []

for district_code in MA_distance_dict['District Code'].tolist():
    
    total_val = enll_total_val = dorm_total_val = 0
    
    for university_name in uni_coord_Public['INSTNM'].tolist():
        
        distance_ij           = distance_district_univeristy_Public(district_code, university_name)
        num_annual_enrollment = uni_coord_Public[uni_coord_Public['INSTNM'] == university_name]['Annual enrollment'].values[0]
        num_dorm_bed          = uni_coord_Public[uni_coord_Public['INSTNM'] == university_name]['Number of dorm beds'].values[0]
        
        total_val      += inverse_distance_weight( distance_ij )
        enll_total_val += enrollment_inverse_distance_weight( distance_ij, num_annual_enrollment )
        dorm_total_val += dormbed_inverse_distance_weight( distance_ij, num_dorm_bed)
        
    inv_distance_arr.append(total_val)
    enll_inv_distance_arr.append(enll_total_val)
    dorm_inv_distance_arr.append(dorm_total_val)
    
MA_distance_dict['Inverse Distance Public'] = inv_distance_arr
MA_distance_dict['Enrollment Inverse Distance Public'] = enll_inv_distance_arr
MA_distance_dict['Dorm Bed Inverse Distance Public'] = dorm_inv_distance_arr


#### 3. Calculate distance to Private nfp
inv_distance_arr      = []
enll_inv_distance_arr = []
dorm_inv_distance_arr = []

for district_code in MA_distance_dict['District Code'].tolist():
    
    total_val = enll_total_val = dorm_total_val = 0
    
    for university_name in uni_coord_Private_nfp['INSTNM'].tolist():
        
        distance_ij           = distance_district_univeristy_Private_nfp(district_code, university_name)
        num_annual_enrollment = uni_coord_Private_nfp[uni_coord_Private_nfp['INSTNM'] == university_name]['Annual enrollment'].values[0]
        num_dorm_bed          = uni_coord_Private_nfp[uni_coord_Private_nfp['INSTNM'] == university_name]['Number of dorm beds'].values[0]
        
        total_val      += inverse_distance_weight( distance_ij )
        enll_total_val += enrollment_inverse_distance_weight( distance_ij, num_annual_enrollment )
        dorm_total_val += dormbed_inverse_distance_weight( distance_ij, num_dorm_bed)
        
    inv_distance_arr.append(total_val)
    enll_inv_distance_arr.append(enll_total_val)
    dorm_inv_distance_arr.append(dorm_total_val)
    
MA_distance_dict['Inverse Distance Private nfp'] = inv_distance_arr
MA_distance_dict['Enrollment Inverse Distance Private nfp'] = enll_inv_distance_arr
MA_distance_dict['Dorm Bed Inverse Distance Private nfp'] = dorm_inv_distance_arr


#### 4. Calculate distance to Land_Grant
inv_distance_arr      = []
enll_inv_distance_arr = []
dorm_inv_distance_arr = []

for district_code in MA_distance_dict['District Code'].tolist():
    
    total_val = enll_total_val = dorm_total_val = 0
    
    for university_name in uni_coord_Land_Grant['INSTNM'].tolist():
        
        distance_ij           = distance_district_univeristy_Land_Grant(district_code, university_name)
        num_annual_enrollment = uni_coord_Land_Grant[uni_coord_Land_Grant['INSTNM'] == university_name]['Annual enrollment'].values[0]
        num_dorm_bed          = uni_coord_Land_Grant[uni_coord_Land_Grant['INSTNM'] == university_name]['Number of dorm beds'].values[0]
        
        total_val      += inverse_distance_weight( distance_ij )
        enll_total_val += enrollment_inverse_distance_weight( distance_ij, num_annual_enrollment )
        dorm_total_val += dormbed_inverse_distance_weight( distance_ij, num_dorm_bed)
        
    inv_distance_arr.append(total_val)
    enll_inv_distance_arr.append(enll_total_val)
    dorm_inv_distance_arr.append(dorm_total_val)
    
MA_distance_dict['Inverse Distance Land Grant'] = inv_distance_arr
MA_distance_dict['Enrollment Inverse Distance Land Grant'] = enll_inv_distance_arr
MA_distance_dict['Dorm Bed Inverse Distance Land Grant'] = dorm_inv_distance_arr


#### 5. Calculate distance to STEM
inv_distance_arr      = []
enll_inv_distance_arr = []
dorm_inv_distance_arr = []

for district_code in MA_distance_dict['District Code'].tolist():
    
    total_val = enll_total_val = dorm_total_val = 0
    
    for university_name in uni_coord_STEM['INSTNM'].tolist():
        
        distance_ij           = distance_district_univeristy_STEM(district_code, university_name)
        num_annual_enrollment = uni_coord_STEM[uni_coord_STEM['INSTNM'] == university_name]['Annual enrollment'].values[0]
        num_dorm_bed          = uni_coord_STEM[uni_coord_STEM['INSTNM'] == university_name]['Number of dorm beds'].values[0]
        
        total_val      += inverse_distance_weight( distance_ij )
        enll_total_val += enrollment_inverse_distance_weight( distance_ij, num_annual_enrollment )
        dorm_total_val += dormbed_inverse_distance_weight( distance_ij, num_dorm_bed)
        
    inv_distance_arr.append(total_val)
    enll_inv_distance_arr.append(enll_total_val)
    dorm_inv_distance_arr.append(dorm_total_val)
    
MA_distance_dict['Inverse Distance STEM'] = inv_distance_arr
MA_distance_dict['Enrollment Inverse Distance STEM'] = enll_inv_distance_arr
MA_distance_dict['Dorm Bed Inverse Distance STEM'] = dorm_inv_distance_arr


#### save this years data in a excel file
MA_distance_dict.to_excel('data/MA_AP_performance/AP_data_combined/MA_distance_dict.xlsx', index=False)

In [54]:
MA_distance_dict

,District Name,District Code,Inverse Distance R1R2,Enrollment Inverse Distance R1R2,Dorm Bed Inverse Distance R1R2,Inverse Distance Public,Enrollment Inverse Distance Public,Dorm Bed Inverse Distance Public,Inverse Distance Private nfp,Enrollment Inverse Distance Private nfp,Dorm Bed Inverse Distance Private nfp,Inverse Distance Land Grant,Enrollment Inverse Distance Land Grant,Dorm Bed Inverse Distance Land Grant,Inverse Distance STEM,Enrollment Inverse Distance STEM,Dorm Bed Inverse Distance STEM
0,Abington,10000,5.742173,113217.532119,35071.588089,17.316076,146209.258415,19738.936728,38.050315,197498.160205,53477.708858,1.119661,21389.723189,8537.261248,6.140646,113694.257088,35207.672552
1,Agawam,50000,4.259957,84369.536271,26822.807686,19.247534,161887.565664,24280.309803,36.316263,153947.897756,41274.155921,1.089071,27508.390439,10673.328163,4.764922,84964.117231,27040.368048
2,Amesbury,70000,4.881265,92335.948273,29213.474888,17.566646,134090.105013,19924.391277,31.722365,205726.665253,41319.510391,1.329773,24366.675339,9873.120424,5.211537,92729.839604,29331.067873
3,Andover,90000,5.897465,115275.780851,35727.560108,18.736678,148918.526677,21110.795019,39.020820,235988.426609,53166.786992,1.251760,23382.478013,9463.319027,6.289450,115746.339650,35854.643254
4,Arlington,100000,8.410371,172279.449988,54114.288808,19.404393,159320.090117,21223.421727,50.726577,290580.664963,77303.949760,1.493855,26476.121346,10965.053231,8.855705,172816.440020,54248.911208
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,Tantasqua,7700000,5.013281,93811.713870,30115.616244,18.390893,155659.672418,24008.573527,35.622710,165714.386126,43858.687410,1.205759,29220.945851,11391.583157,5.622438,94547.476298,30296.981200
218,Triton,7730000,5.090309,97963.676272,30690.200516,17.466648,135924.537762,19555.661808,33.866870,207493.196931,45035.341613,1.243468,22947.499722,9280.496422,5.432690,98372.542760,30809.929372
219,Wachusett,7750000,5.640881,99323.789902,31578.929877,19.239305,156707.553686,22970.364011,37.340195,187018.631313,47651.230826,1.141326,25704.013619,10080.805304,6.425274,100288.244063,31739.266261
220,Quaboag Regional,7780000,4.927755,93157.520341,29956.704080,18.403836,155590.623474,24241.941837,35.492845,165185.700351,43281.888243,1.212916,29837.433483,11634.616407,5.517016,93868.089505,30137.043753


# Load AP data from 2018 to 2022, then concat the distance data (shown above) to the AP data.

In [58]:
for year in [18, 19, 20, 21, 22]:

    MA_median_income = pd.read_excel('data/MA_AP_performance/MA_median_household_income.xlsx')
    MA_district_population = pd.read_excel('data/MA_AP_performance/MA_district_population.xlsx')
    MA_expenditure_year = pd.read_excel('data/MA_AP_performance/PerPupilExpenditures/PerPupilExpenditures_20%s.xlsx'%year)
    
    
    MA_AP_performance_year = pd.read_excel('data/MA_AP_performance/AP_performance_18_22.xlsx', sheet_name='20%s-%s'%(year, year+1))
    MA_AP_performance_year = MA_AP_performance_year[MA_AP_performance_year['District Code'].isin(MA_distance_dict['District Code'])]
    MA_AP_performance_year = MA_AP_performance_year[MA_AP_performance_year['District Code'].isin(MA_expenditure_year['District Code'])]
    MA_AP_performance_year = MA_AP_performance_year.sort_values(by='District Code').reset_index(drop=True)
    
    
    
    #### delete redundant columns in the AP_performance excel sheet 
    MA_all_data_year = MA_AP_performance_year.drop(columns=['Score=1', 'Score=1', 'Score=2', 'Score=3', 'Score=4', 'Score=5', '% Score 1-2'])
    MA_all_data_year.insert(2, 'Year', [2000+year] * MA_all_data_year.shape[0])
    MA_all_data_year = MA_all_data_year.sort_values(by='District Code').reset_index(drop=True)
    
    
    #### Add a colum for district population
    MA_district_population = MA_district_population[MA_district_population['District Code'].isin(MA_all_data_year['District Code'])]
    MA_district_population = MA_district_population.sort_values(by='District Code').reset_index(drop=True)
    MA_all_data_year['Population'] = MA_district_population['Population']
    
    #### Add a colum for district median household income
    MA_median_income = MA_median_income[MA_median_income['District Code'].isin(MA_all_data_year['District Code'])]
    MA_median_income = MA_median_income.sort_values(by='District Code').reset_index(drop=True)
    MA_all_data_year['Median Household Income'] = MA_median_income['Median Household Income']
    
    #### Add a colum for district total expenditure per pupil
    MA_expenditure_year = MA_expenditure_year[MA_expenditure_year['District Code'].isin(MA_all_data_year['District Code'])]
    MA_expenditure_year = MA_expenditure_year.sort_values(by='District Code').reset_index(drop=True)
    expenditure_arr = MA_expenditure_year['Total Expenditures per Pupil'].tolist()
    expenditure_arr = np.array([float(str(x).replace('$', '').replace(',', '').strip()) if isinstance(x, str) else float(x) for x in expenditure_arr])
    MA_all_data_year['Total Expenditures per Pupil'] = expenditure_arr
    
    #### save this years data in a excel file
    MA_all_data_year = pd.concat([MA_all_data_year, MA_distance_dict.iloc[:, 2:]], axis=1)
    MA_all_data_year.to_excel('data/MA_AP_performance/AP_data_combined/AP_data_combined_20%s.xlsx'%year, index=False)

# Combine all five year data in one single Excel file

In [65]:
data18 = pd.read_excel('data/MA_AP_performance/AP_data_combined/AP_data_combined_2018.xlsx').sort_values(by='District Code')
data19 = pd.read_excel('data/MA_AP_performance/AP_data_combined/AP_data_combined_2019.xlsx').sort_values(by='District Code')
data20 = pd.read_excel('data/MA_AP_performance/AP_data_combined/AP_data_combined_2020.xlsx').sort_values(by='District Code')
data21 = pd.read_excel('data/MA_AP_performance/AP_data_combined/AP_data_combined_2021.xlsx').sort_values(by='District Code')
data22 = pd.read_excel('data/MA_AP_performance/AP_data_combined/AP_data_combined_2022.xlsx').sort_values(by='District Code')

combined_data_18_22 = pd.concat([data18, data19, data20, data21, data22], ignore_index=True)

combined_data_18_22.to_csv("data/MA_AP_performance/AP_data_combined_18_22.csv", index=False)

with pd.ExcelWriter("data/MA_AP_performance/AP_data_combined_18_22.xlsx") as writer:
    combined_data_18_22.to_excel(writer, sheet_name="2018-22", index=False)
    data18.to_excel(writer, sheet_name="2018-19", index=False)
    data19.to_excel(writer, sheet_name="2019-20", index=False)
    data20.to_excel(writer, sheet_name="2020-21", index=False)
    data21.to_excel(writer, sheet_name="2021-22", index=False)
    data22.to_excel(writer, sheet_name="2022-23", index=False)